## Imports for the Project and Setup

In [ ]:
# Full reset of the camera
!echo 'dlinano' | sudo -S systemctl restart nvargus-daemon && printf '\n'
# Check device number
#!ls -ltrh /dev/video*

In [ ]:
import torch
import torchvision
from torchvision import transforms , datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary # for model summary
import matplotlib.pyplot as plt # for ploting our data and showing images
import numpy as np
import cv2
import dataHelper as dH
import os
import time
import ipywidgets
from IPython.display import display, Javascript
from jupyter_clickable_image_widget import ClickableImageWidget
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera
%matplotlib inline

In [ ]:
camera = CSICamera(width=224, height=224)

## Model Setup

Here we are letting pytorch know to use the gpu and we are also importing a pretrained resnet18 model that we are modifying to return only 2 outputs. One for X and one for Y.

In [ ]:
device = torch.device('cuda')
output_dim = 2
# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)
model = model.to(device)

## Model Summary
This is the inner workings of resnet18.

In [ ]:
summary(model, input_size=(3,224,224))

## Data Collection

### Files Structure Setup

In [ ]:
path_raw = os.getcwd() + '/raw_datasets'
try:
    os.mkdir(path_raw)
except OSError:
    pass
else:
    pass

In [ ]:
#config
datacollect = False
dataSetSize = 25
FPScollect = 30
SEC_delay = 2


loopcounter = 0
collectedNum = 0

if datacollect == True:
    image_widget = ipywidgets.Image(format='jpeg')
    captured_frame = ipywidgets.Image(format='jpeg')
    
    camera.unobserve_all()
    print('Get Ready!')
    frame = camera.read()
    image_widget.value = bgr8_to_jpeg(frame)
    captured_frame.value = bgr8_to_jpeg(frame)
    #plt.imshow(bgr8_to_jpeg(frame))
    time.sleep(6)
    display(ipywidgets.HBox([image_widget, captured_frame]))
    print('Begin!')
    
    while True:
        if collectedNum == dataSetSize:
            break
        frame = camera.read()
        #plt.imshow(frame)
        image_widget.value = bgr8_to_jpeg(frame)
        if loopcounter % (FPScollect * SEC_delay) == 0:
            # write raw images to file 
            captured_frame.value = bgr8_to_jpeg(frame)
            dH.write_raw_frame(path_raw, frame, collectedNum)
            collectedNum += 1
        loopcounter += 1
print('Done!')

## Data Labeling

An I oop... data is labeled on aonther PC because some libraries are broke

## Model Training

In [ ]:
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

EPOCHS = 3

